In [1]:
import pandas as pd
import datetime
import locale 

locale.setlocale(locale.LC_ALL, '')

'Russian_Russia.1251'

In [2]:
def merge_columns(col):
    if col.name == "Account":
        return col.iloc[-1]
    elif col.isna().all():
        return float("nan")
    elif "стоп" in col.values:
        return "стоп"
    elif "end" in col.values:
        return "end"
    else:
        return pd.to_numeric(
            col.fillna(0.0).astype(str).str.replace(r'[ в ноль]', '', regex=True).str.replace(',', '.'),
        ).sum()

def fill_missing_cells(row):
    prev_non_zero = None
    for i in range(len(row)):
        val = row.iloc[i]
        
        if val in ["стоп", "end"]:
            row.iloc[i] = float("nan")
            val = row.iloc[i]
        if pd.isna(val):
            prev_non_zero = None
            continue
        
        # Значение из одной клетки может переносится на несколько нулей подряд, что полезно при подсчете
        if val == 0:
            if prev_non_zero != None:
                row.iloc[i] = prev_non_zero
        else:
            prev_non_zero = val
    return row


In [3]:
fin_data = pd.read_csv('financial_data.csv')
fin_data.drop("Причина дубля", inplace=True, axis=1)
fin_data = fin_data.groupby("id").agg(merge_columns)


# 2 первых и 2 последних месяца включены в подсчет по месяцам для справки,
# так как по тз не требуется их подсчет, но информация по ним есть
# 2 дополнительных столбца нужны лишь фиктивно для подсчета двух месяцев до 2023,
# так как есть смысл их тоже посчитать для демонстрации, на случай если работники только начали работать
fin_data.insert(0, "Октябрь 2022", float("nan"))
fin_data.insert(0, "Сентябрь 2022", float("nan"))

fin_data.rename(columns=lambda name: name if name == "Account" else datetime.datetime.strptime(name, '%B %Y'), inplace=True)

prlngs = pd.read_csv('prolongations.csv')
prlngs = prlngs.apply(
    lambda col: col if col.name != "month" else [datetime.datetime.strptime(month, '%B %Y') for month in col.values]
)

# Очистка от тех, у которых после предыдущего продления до текущего в fin_data есть стоп/end
for [am_full_name, id], prlngs_months in prlngs.groupby(["AM", "id"]):
    i_prev_month = 2 # берем месяц, начиная с которого есть данные
    prev_month: str = fin_data.columns[i_prev_month]
    
    for idx_in_prlngs, cur_month in prlngs_months.loc[:, "month"].items():
        
        i_cur_month = list(fin_data.columns).index(cur_month)

        cur_prlng_segment = fin_data.iloc[:, i_prev_month:i_cur_month+1].loc[id] # берем отрезок от предыдущего до текущего
        if cur_prlng_segment.isin(["стоп", "end"]).any():
            prlngs.drop(idx_in_prlngs, inplace=True)
        i_prev_month = i_cur_month+1
        prev_month = cur_month
        
fin_data = pd.DataFrame(fin_data.apply(fill_missing_cells, axis=1))

print(prlngs.to_string())
print()
print(fin_data.to_string())

       id      month                             AM
0      42 2022-11-01   Васильев Артем Александрович
1     453 2022-11-01   Васильев Артем Александрович
2     548 2022-11-01      Михайлов Андрей Сергеевич
3      87 2022-11-01  Соколова Анастасия Викторовна
4     429 2022-11-01  Соколова Анастасия Викторовна
5     600 2022-11-01   Васильев Артем Александрович
8     665 2022-11-01   Васильев Артем Александрович
9     586 2022-11-01        Иванова Мария Сергеевна
10    637 2022-11-01  Соколова Анастасия Викторовна
11    419 2022-11-01      Михайлов Андрей Сергеевич
12    578 2022-11-01    Попова Екатерина Николаевна
13    579 2022-11-01   Васильев Артем Александрович
14    671 2022-11-01  Соколова Анастасия Викторовна
15    633 2022-11-01   Васильев Артем Александрович
16    645 2022-11-01  Соколова Анастасия Викторовна
17    369 2022-11-01   Васильев Артем Александрович
18    547 2022-11-01      Михайлов Андрей Сергеевич
19    592 2022-11-01   Васильев Артем Александрович
20    666 20

In [4]:
# Подготавливаем shipments, чтобы потом создать сводную таблицу
shipments = []

for [am_full_name, cur_month], ids in prlngs.groupby(["AM", "month"])["id"]:
    new_row = [am_full_name, cur_month]
    
    i_prlng_in_fin_data = list(fin_data.columns).index(cur_month)
    
    conv_for_prlngs_in_fst_month = (
        fin_data.index.isin(ids) & 
        fin_data.iloc[:, i_prlng_in_fin_data-1].notna()
    )

    prev_month_prlng_for_fst = fin_data.iloc[:, i_prlng_in_fin_data-1].loc[conv_for_prlngs_in_fst_month]
    cur_month_prlng_for_fst = fin_data.iloc[:, i_prlng_in_fin_data].loc[conv_for_prlngs_in_fst_month]
    
    prev_month_non_prlng_for_fst = fin_data.iloc[:, i_prlng_in_fin_data-1][
        (fin_data['Account'] == am_full_name) &
        fin_data.iloc[:, i_prlng_in_fin_data-1].notna() &
        fin_data.iloc[:, i_prlng_in_fin_data].isna() &
        ~fin_data.iloc[:, i_prlng_in_fin_data].isin(ids)
    ]
    prev_month_all_for_fst = prev_month_prlng_for_fst.sum()+prev_month_non_prlng_for_fst.sum()
    cur_month_prlng_for_fst = cur_month_prlng_for_fst.sum()
    fst_month_coef = cur_month_prlng_for_fst/prev_month_all_for_fst
    
    
    conv_for_prlngs_in_snd_month = (
        fin_data.index.isin(ids) & 
        fin_data.iloc[:, i_prlng_in_fin_data-1].isna() &
        fin_data.iloc[:, i_prlng_in_fin_data-2].notna()
    )
    
    prev_month_prlng_for_snd = fin_data.iloc[:, i_prlng_in_fin_data-2].loc[conv_for_prlngs_in_snd_month]
    cur_month_prlng_for_snd = fin_data.iloc[:, i_prlng_in_fin_data].loc[conv_for_prlngs_in_snd_month]
    
    prev_month_non_prlng_for_snd = fin_data.iloc[:, i_prlng_in_fin_data-2][
        (fin_data['Account'] == am_full_name) &
        fin_data.iloc[:, i_prlng_in_fin_data-1].isna() &
        fin_data.iloc[:, i_prlng_in_fin_data-2].notna() &
        ~fin_data.iloc[:, i_prlng_in_fin_data].isin(ids)
    ]
    prev_month_all_for_snd = prev_month_prlng_for_snd.sum()+prev_month_non_prlng_for_snd.sum()
    cur_month_prlng_for_snd = cur_month_prlng_for_snd.sum()
    snd_month_coef = cur_month_prlng_for_snd/prev_month_all_for_snd
    
    
    new_row.extend([
        cur_month_prlng_for_fst, prev_month_all_for_fst, cur_month_prlng_for_snd, prev_month_all_for_snd, fst_month_coef, snd_month_coef
    ])
    shipments.append(new_row)
df_shipments = pd.DataFrame(shipments, columns=[
    "AM", "month", "cur_month_prlng_for_fst", "prev_month_all_for_fst", "cur_month_prlng_for_snd", "prev_month_all_for_snd", "fst_month_coef", "snd_month_coef"
])

print(df_shipments.to_string())


                               AM      month  cur_month_prlng_for_fst  prev_month_all_for_fst  cur_month_prlng_for_snd  prev_month_all_for_snd  fst_month_coef  snd_month_coef
0    Васильев Артем Александрович 2022-11-01                     0.00                    0.00                      0.0                    0.00             NaN             NaN
1    Васильев Артем Александрович 2022-12-01                888572.68              1246638.61                      0.0                    0.00        0.712775             NaN
2    Васильев Артем Александрович 2023-01-01                827225.00              1539702.00                      0.0               494412.00        0.537263        0.000000
3    Васильев Артем Александрович 2023-02-01                899803.00               838615.00                      0.0               827785.00        1.072963        0.000000
4    Васильев Артем Александрович 2023-03-01               1161675.45              1731260.47                      0.0       

C:\Users\kozmi\AppData\Local\Temp\ipykernel_6680\4225964824.py:25: RuntimeWarning: invalid value encountered in scalar divide
  fst_month_coef = cur_month_prlng_for_fst/prev_month_all_for_fst
C:\Users\kozmi\AppData\Local\Temp\ipykernel_6680\4225964824.py:45: RuntimeWarning: invalid value encountered in scalar divide
  snd_month_coef = cur_month_prlng_for_snd/prev_month_all_for_snd


In [5]:
shipments_pivot = df_shipments.pivot(
    index="AM", columns="month", values=[
        "cur_month_prlng_for_fst", "prev_month_all_for_fst",
        "cur_month_prlng_for_snd", "prev_month_all_for_snd", 
        "fst_month_coef", "snd_month_coef"
    ]
)


shipments_pivot.loc["Итог за мес"] = shipments_pivot.agg(
    lambda col: col.sum() if col.name[0] not in ["fst_month_coef", "snd_month_coef"] else float("nan")
)
for date in shipments_pivot.columns.levels[1]: # type: ignore
    shipments_pivot.loc["Итог за мес", ('fst_month_coef', date)] = (shipments_pivot.loc["Итог за мес", ('cur_month_prlng_for_fst', date)] / 
                                                             shipments_pivot.loc["Итог за мес", ('prev_month_all_for_fst', date)]) # type: ignore
    
    shipments_pivot.loc["Итог за мес", ('snd_month_coef', date)] = (shipments_pivot.loc["Итог за мес", ('cur_month_prlng_for_snd', date)] / 
                                                             shipments_pivot.loc["Итог за мес", ('prev_month_all_for_snd', date)]) # type: ignore


year = None
cur_year_prlng_for_fst, prev_year_all_for_fst, cur_year_prlng_for_snd, prev_year_all_for_snd = 0.0, 0.0, 0.0, 0.0
for date in shipments_pivot.columns.levels[1]: # type: ignore
    new_year = str(date)[:4]
    if new_year != year:
        cur_year_prlng_for_fst, prev_year_all_for_fst, cur_year_prlng_for_snd, prev_year_all_for_snd = 0.0, 0.0, 0.0, 0.0
        year = new_year
    cur_year_prlng_for_fst += shipments_pivot.loc["Итог за мес", ("cur_month_prlng_for_fst", date)] # type: ignore
    prev_year_all_for_fst += shipments_pivot.loc["Итог за мес", ("prev_month_all_for_fst", date)] # type: ignore
    cur_year_prlng_for_snd += shipments_pivot.loc["Итог за мес", ("cur_month_prlng_for_snd", date)] # type: ignore
    prev_year_all_for_snd += shipments_pivot.loc["Итог за мес", ("prev_month_all_for_snd", date)] # type: ignore
    if str(date).endswith("12-01 00:00:00"):
        shipments_pivot.loc["Итог за год", ("cur_month_prlng_for_fst", date)] = cur_year_prlng_for_fst
        shipments_pivot.loc["Итог за год", ("prev_month_all_for_fst", date)] = prev_year_all_for_fst
        shipments_pivot.loc["Итог за год", ("cur_month_prlng_for_snd", date)] = cur_year_prlng_for_snd
        shipments_pivot.loc["Итог за год", ("prev_month_all_for_snd", date)] = prev_year_all_for_snd
        shipments_pivot.loc["Итог за год", ('fst_month_coef', date)] = cur_year_prlng_for_fst/prev_year_all_for_fst
        shipments_pivot.loc["Итог за год", ('snd_month_coef', date)] = cur_year_prlng_for_snd/prev_year_all_for_snd

print(shipments_pivot.to_string())


                              cur_month_prlng_for_fst                                                                                                                                                           prev_month_all_for_fst                                                                                                                                                            cur_month_prlng_for_snd                                                                                                                                                prev_month_all_for_snd                                                                                                                                                         fst_month_coef                                                                                                                                                snd_month_coef                                                                                                    

C:\Users\kozmi\AppData\Local\Temp\ipykernel_6680\3507914270.py:14: RuntimeWarning: invalid value encountered in scalar divide
  shipments_pivot.loc["Итог за мес", ('fst_month_coef', date)] = (shipments_pivot.loc["Итог за мес", ('cur_month_prlng_for_fst', date)] /
C:\Users\kozmi\AppData\Local\Temp\ipykernel_6680\3507914270.py:17: RuntimeWarning: invalid value encountered in scalar divide
  shipments_pivot.loc["Итог за мес", ('snd_month_coef', date)] = (shipments_pivot.loc["Итог за мес", ('cur_month_prlng_for_snd', date)] /
C:\Users\kozmi\AppData\Local\Temp\ipykernel_6680\3507914270.py:38: RuntimeWarning: invalid value encountered in scalar divide
  shipments_pivot.loc["Итог за год", ('snd_month_coef', date)] = cur_year_prlng_for_snd/prev_year_all_for_snd


In [6]:
from openpyxl.utils import get_column_letter
from openpyxl.styles import numbers

def create_prolongation_summary_in_excel(writer):
    shipments_pivot.to_excel(writer, sheet_name='prolongation_summary')
    workbook = writer.book
    worksheet = workbook['prolongation_summary']

    for col_idx, column in enumerate(worksheet.columns, start=1):
        cell = worksheet.cell(row=2, column=col_idx)
        cell.number_format = numbers.FORMAT_DATE_XLSX14
        column_letter = get_column_letter(column[0].column)
        if col_idx == 1:
            worksheet.column_dimensions[column_letter].width = 30
            worksheet.freeze_panes = 'B1'
        else:
            worksheet.column_dimensions[column_letter].width = 10

file_path = "prolongation_summary.xlsx"

try:
    with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists="replace") as writer:
        create_prolongation_summary_in_excel(writer)
except (ValueError, FileNotFoundError):
    with pd.ExcelWriter(file_path, engine='openpyxl', mode='w') as writer:
        create_prolongation_summary_in_excel(writer)
